In [1]:
# Dependencies
import pandas as pd
import pymongo
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#def scrape_mars_news(browser):
executable_path = {'executable_path': ChromeDriverManager().install()}
print(executable_path)

{'executable_path': 'C:\\Users\\pravi\\.wdm\\drivers\\chromedriver\\win32\\105.0.5195\\chromedriver.exe'}


In [3]:
#pip install selenium

In [4]:
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
listing = {}
listings_df = pd.DataFrame()

In [6]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [7]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

### NASA Mars News

In [8]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('div', class_='list_text')
results

[<div class="list_text">
 <div class="list_date">October 2, 2022</div>
 <div class="content_title">From JPL's Mailroom to Mars and Beyond</div>
 <div class="article_teaser_body">Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">October 1, 2022</div>
 <div class="content_title">NASA's New Mars Rover Will Use X-Rays to Hunt Fossils</div>
 <div class="article_teaser_body">PIXL, an instrument on the end of the Perseverance rover's arm, will search for chemical fingerprints left by ancient microbes.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">September 30, 2022</div>
 <div class="content_title">AI Is Helping Scientists Discover Fresh Craters on Mars</div>
 <div class="article_teaser_body">It's the first time machine learning has been used to find previously unknown craters on the Red Planet.</div

In [9]:
# Loop through returned results
for result in results:
    
    # Error handling
    try:
        # Identify and return title of listing
        date = result.find('div', class_='list_date').text
        title = result.find('div', class_='content_title').text
        # Identify and return the explanation of the listing
        paragraph = result.find('div', class_='article_teaser_body').text
       

        # Run only if date and title and paragraph are available
        if (date and title and paragraph):
            # Print results
            print('-------------')
            print(date)
            print(title)
            print(paragraph)
            
            listing = {'date':date, 'title': title, 'paragraph':paragraph}
            listings_df = pd.concat([listings_df, listing], axis=0)
        
    except:       
            print('Data not found')


-------------
October 2, 2022
From JPL's Mailroom to Mars and Beyond
Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.
Data not found
-------------
October 1, 2022
NASA's New Mars Rover Will Use X-Rays to Hunt Fossils
PIXL, an instrument on the end of the Perseverance rover's arm, will search for chemical fingerprints left by ancient microbes.
Data not found
-------------
September 30, 2022
AI Is Helping Scientists Discover Fresh Craters on Mars
It's the first time machine learning has been used to find previously unknown craters on the Red Planet.
Data not found
-------------
September 24, 2022
NASA Engineers Checking InSight's Weather Sensors
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.
Data not found
-------------
September 21, 2022
What's Mars Solar Conjunction, and Why Does It Matter?
NAS

In [10]:
browser.quit()
#     return(listings_df)

### JPL Mars Space Images—Featured Image

In [11]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [12]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [13]:
# HTML Object 
html_image = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_image, 'html.parser')

relative_image_path = soup.find_all('img')[1]["src"]

featured_image_url = url + relative_image_path
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [14]:
browser.quit()

### Mars Facts

In [15]:
url = 'https://galaxyfacts-mars.com'
tables = pd.read_html(url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [16]:
type(tables)

list

In [17]:
table_df = tables[0]
table_df 

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [18]:
# Drop the earth data from the dataframe
table_df = table_df.drop([2], axis=1)
table_df = table_df.iloc[1: , :]
table_df.columns=['Description', 'Values']
table_df.set_index('Description', inplace=True)
table_df

,Values
Description,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-87 to -5 °C


In [19]:
# Convert the table back to HTML
html_table = table_df.to_html()
html_table = html_table.replace('\n','')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [20]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://marshemispheres.com/'
browser.visit(url)

In [21]:
# Get a List of All the Hemispheres
# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://marshemispheres.com/'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
  
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [22]:
browser.quit()